In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
fileid='1LpuTmUqO0qLuiZt-nNUOY1Urbye2K5f-'
downloaded=drive.CreateFile({'id':fileid})
downloaded.GetContentFile('FAKENEWSDETECT_DATASET.csv')


In [0]:

import pandas as pd    #data processing
import numpy as np     #linear algebra
import matplotlib.pyplot as plt   #plotting graphs
from mpl_toolkits.mplot3d import Axes3D   #3d image for an object
from sklearn.preprocessing import StandardScaler    #Standardization of data
import os

In [0]:
#nRowsRead=1000
df1=pd.read_csv('FAKENEWSDETECT_DATASET.csv',nrows=5000)


In [0]:
from sklearn.model_selection import train_test_split
df2 = df1[pd.notnull(df1['fake'])]
df2 = df2[pd.notnull(df2['news'])]
y=(df2['fake']==0).astype(int)


In [9]:
df1['fake'].unique().tolist()


[nan, 1.0, 0.0]

In [11]:
print(sum(df2['fake']==0))

2437


In [0]:

x_aug =df2['news']
y_aug =y


In [14]:
print(df2.shape)
print(y.shape)


(2477, 3)
(2477,)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(x_aug, y_aug, test_size=0.25, random_state=42)

In [16]:
X_train.head()

1737    'Big Daddy' laps up Cipla after Q1 nos beat fo...
3281     Kartarpur Corridor: In half-hearted attempt, ...
3863     In Japan, the business of watching whales is ...
4477     Pro-Balochistan activists disrupt Imran Khan'...
2144     IMF approves $6 billion loan for cash-strappe...
Name: news, dtype: object

In [17]:
X_train.shape

(1857,)

In [18]:
y_train.head()

1737    1
3281    1
3863    1
4477    1
2144    1
Name: fake, dtype: int64

In [19]:
y_train.shape

(1857,)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Initialize the `tfidf_vectorizer` 
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7) 

# Fit and transform the training data 
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 

# Transform the test set 
tfidf_test = tfidf_vectorizer.transform(X_test)


In [22]:
print(tfidf_train.shape)

(1857, 7788)


In [0]:
from sklearn.feature_extraction.text import CountVectorizer
# Initialize the `count_vectorizer` 
count_vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the training data 
count_train = count_vectorizer.fit_transform(X_train) 

# Transform the test set 
count_test = count_vectorizer.transform(X_test)



In [24]:
# Get the feature names of `tfidf_vectorizer` 
print(tfidf_vectorizer.get_feature_names()[-10:])

# Get the feature names of `count_vectorizer` 
print(count_vectorizer.get_feature_names()[:10])

['zone', 'zoo', 'zpdqb', 'zq4vmb6lyuxzq3j50_bybfzti', 'zqdp1decxsqzqwmf3f9yhthjoi3xjjwg', 'ztgrisjaeespgmbngpxgg6newkmtn', 'zu4azo', 'zv1cgf4kmbql76abh2q', 'zyspbx0x9fiol4khoxtvvmifenu2cimc', 'zzjdhsfe']
['00', '000', '001', '00r5ajnpxjwifn4kz_hizgew7b2e4iv3', '01', '02', '02t044543z_1216169853_', '02t162600z_1842575144_', '03', '03am']


In [25]:
count_df = pd.DataFrame(count_train.A, columns=count_vectorizer.get_feature_names())
tfidf_df = pd.DataFrame(tfidf_train.A, columns=tfidf_vectorizer.get_feature_names())
difference = set(count_df.columns) - set(tfidf_df.columns)
difference
print(count_df.equals(tfidf_df))
count_df.head()

False


,00,000,001,00r5ajnpxjwifn4kz_hizgew7b2e4iv3,01,02,02t044543z_1216169853_,02t162600z_1842575144_,03,03am,04,04t095756z_1107249886__0,05,053,05t014353z_41458599_rc,06,07,08,09,0beevimpbcirlxxw0zx,0c_ivwajd3kqfvcf1qkmdyozlgj1zx8m,0ckyipwrtoadh5rk9ciakv3r_0avwu0q,0edoc9x,0et4gmolx,0jjo5qixmmdajlhaxtytvs3tsmghaa8b,0tlkd1x_bzd7bvdrzd,0tqxphipbj916f58livrxsqxrl3hxomd,0ut,10,100,1000,1000_1,1000_2,1000_3,1000__10_,1000__3_,1000__4_,1000__5_,1000__7_,11,...,youin,young,younger,youngsters,youth,yuvraj,yvadkdy2yt4awoo9rwmuqv0dsalf1y6x,yvbkqs80inbltia6ofrahulmrwhp1tle,yxqls9izqrufx40,yz24yui5,yz8r7ze6qiawnke9a7kwm83dkd2espe_,yzsd84umdpqo_xqk6u6szjjiaimllq8a,z6vi3an3nxhwco42fxybd3appyxvfxjz,z6zl,z8qlqycbymsv04stuvo_sntvcswkefkf,zahr,zara,zardari,zd6wwiycswef7jn,zealand,zemxaiqo8gq8ftbt4rzxokor5fgwh,zenfdegii,zero,zhzdhdz,zio5i,zjoclca8rmiqb_g3fg46fh8nhknuruc,zjx,zne5t7trrgxeheucepn,zoella,zoetis,zone,zoo,zpdqb,zq4vmb6lyuxzq3j50_bybfzti,zqdp1decxsqzqwmf3f9yhthjoi3xjjwg,ztgrisjaeespgmbngpxgg6newkmtn,zu4azo,zv1cgf4kmbql76abh2q,zyspbx0x9fiol4khoxtvvmifenu2cimc,zzjdhsfe
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
tfidf_df.head()

,00,000,001,00r5ajnpxjwifn4kz_hizgew7b2e4iv3,01,02,02t044543z_1216169853_,02t162600z_1842575144_,03,03am,04,04t095756z_1107249886__0,05,053,05t014353z_41458599_rc,06,07,08,09,0beevimpbcirlxxw0zx,0c_ivwajd3kqfvcf1qkmdyozlgj1zx8m,0ckyipwrtoadh5rk9ciakv3r_0avwu0q,0edoc9x,0et4gmolx,0jjo5qixmmdajlhaxtytvs3tsmghaa8b,0tlkd1x_bzd7bvdrzd,0tqxphipbj916f58livrxsqxrl3hxomd,0ut,10,100,1000,1000_1,1000_2,1000_3,1000__10_,1000__3_,1000__4_,1000__5_,1000__7_,11,...,youin,young,younger,youngsters,youth,yuvraj,yvadkdy2yt4awoo9rwmuqv0dsalf1y6x,yvbkqs80inbltia6ofrahulmrwhp1tle,yxqls9izqrufx40,yz24yui5,yz8r7ze6qiawnke9a7kwm83dkd2espe_,yzsd84umdpqo_xqk6u6szjjiaimllq8a,z6vi3an3nxhwco42fxybd3appyxvfxjz,z6zl,z8qlqycbymsv04stuvo_sntvcswkefkf,zahr,zara,zardari,zd6wwiycswef7jn,zealand,zemxaiqo8gq8ftbt4rzxokor5fgwh,zenfdegii,zero,zhzdhdz,zio5i,zjoclca8rmiqb_g3fg46fh8nhknuruc,zjx,zne5t7trrgxeheucepn,zoella,zoetis,zone,zoo,zpdqb,zq4vmb6lyuxzq3j50_bybfzti,zqdp1decxsqzqwmf3f9yhthjoi3xjjwg,ztgrisjaeespgmbngpxgg6newkmtn,zu4azo,zv1cgf4kmbql76abh2q,zyspbx0x9fiol4khoxtvvmifenu2cimc,zzjdhsfe
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.031340,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.036066,0.0,0.099959,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.033722,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083165,0.0,0.0,0.0,0.0,0.0,0.028988,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
X_train.head()

1737    'Big Daddy' laps up Cipla after Q1 nos beat fo...
3281     Kartarpur Corridor: In half-hearted attempt, ...
3863     In Japan, the business of watching whales is ...
4477     Pro-Balochistan activists disrupt Imran Khan'...
2144     IMF approves $6 billion loan for cash-strappe...
Name: news, dtype: object

In [0]:

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [29]:
print(y_train.unique().tolist())

[1, 0]


In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
knn = KNeighborsClassifier()
#y_train_vec = tfidf_vectorizer.fit_transform(y_train)
knn.fit(tfidf_train,y_train)
prediction = knn.predict(tfidf_test)
#y_test_vec = tfidf_vectorizer.transform(y_test)
score = accuracy_score(y_test, prediction)
print("test accuracy:   %0.3f" % score)


test accuracy:   0.992


In [32]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB() 
#y_train_vec = tfidf_vectorizer.fit_transform(y_train)
#y_test_vec = tfidf_vectorizer.transform(y_test)
clf.fit(tfidf_train, y_train)
pred = clf.predict(tfidf_test)
score = accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)
#print(len(pred))
#print(sum(y_test==1))
#print(sum(pred==1))

accuracy:   1.000


In [33]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf.fit(tfidf_train, y_train)
prediction = rf.predict(tfidf_test)

print('With Random Forest accuracy is: ',accuracy_score(y_test,prediction)) # accuracy


With Random Forest accuracy is:  1.0


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [36]:
df2.head()

,news,fake,source
1,Majority of farm animal vets report being inju...,1.0,BVA
3,Vets and animal welfare experts encouraged to ...,1.0,BVA
5,Call for top pics as BVA launches 2019 Veterin...,1.0,BVA
7,NI vets applaud swift action as African Swine ...,1.0,BVA
9,Expert speakers to go ‘behind the headlines’ a...,1.0,BVA


In [0]:
def clean_data(dataframe):
# Drop duplicate rows
    dataframe.drop_duplicates(subset='news', inplace=True)
    
    # Remove punctation
    dataframe['news'] = dataframe['news'].str.replace('[^\w\s]',' ')
# Remove numbers 
    dataframe['news'] = dataframe['news'].str.replace('[^A-Za-z]',' ')
# Make sure any double-spaces are single 
    dataframe['news'] = dataframe['news'].str.replace('  ',' ')
    dataframe['news'] = dataframe['news'].str.replace('  ',' ')
# Transform all text to lowercase
    dataframe['news'] = dataframe['news'].str.lower()
    
    print("New shape:", dataframe.shape)
    return dataframe.head()

In [38]:
clean_data(df2)

New shape: (1048, 3)


,news,fake,source
1,majority of farm animal vets report being inju...,1.0,BVA
3,vets and animal welfare experts encouraged to ...,1.0,BVA
5,call for top pics as bva launches veterinary ...,1.0,BVA
7,ni vets applaud swift action as african swine ...,1.0,BVA
9,expert speakers to go behind the headlines at ...,1.0,BVA


In [0]:
from sklearn.model_selection import train_test_split
y = df2['news'] 
#X_train, X_test, y_train, y_test = train_test_split(ds['Body'], y, test_size=0.33, random_state=53)

In [40]:
# Initialize the `tfidf_vectorizer` 
#tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7) 

# Fit and transform the training data 
#tfidf_evaluate = tfidf_vectorizer.fit_transform(y) 
print(y.shape)
# Transform the test set 
tfidf_evaluate = tfidf_vectorizer.transform(y)
print(tfidf_evaluate.shape)

(1048,)
(1048, 7788)


In [41]:
#from sklearn.neighbors import KNeighborsClassifier
#knn = KNeighborsClassifier()
#y_train_vec = tfidf_vectorizer.fit_transform(y_train)
#knn.fit(tfidf_train,y_train)
prediction = knn.predict(tfidf_evaluate)
#y_test_vec = tfidf_vectorizer.transform(y_test)
#score = accuracy_score(y_test, prediction)
#print("accuracy:   %0.3f" % score)
print(prediction)

[0 0 0 ... 1 1 1]


In [42]:
#from sklearn.naive_bayes import MultinomialNB
#clf = MultinomialNB() 
#y_train_vec = tfidf_vectorizer.fit_transform(y_train)
#y_test_vec = tfidf_vectorizer.transform(y_test)
#clf.fit(tfidf_train, y_train)
pred = clf.predict(tfidf_evaluate)
print(pred)
#score = accuracy_score(y_test, pred)
#print("accuracy:   %0.3f" % score)

[0 0 0 ... 1 1 1]


In [0]:
predictrf = rf.predict(tfidf_evaluate)

In [44]:
print(predictrf)

[0 0 0 ... 1 1 1]
